#Learning Objectives:

By the end of this lesson the student will:
- Merge and concatenate data frames and arrays
- Reshape and pivot data frames

In [21]:
import pandas as pd
import numpy as np

## Combining and Merging data sets

In [5]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [6]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [7]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


Note that merge works like an inner.join. Only uses keys common for both data frames by default. Otherwise need to specify how.

In [8]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,NaN
7,NaN,d,2


You can also merge on multiple keys

In [10]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'],
'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [12]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'],
'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [13]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


In [14]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


You can also merge on indexes

In [16]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [18]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [19]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


## Challenge! Using the following two dataframes, do an outer merge:

In [23]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2': [2000, 2001, 2002, 2001, 2002],
'data': np.arange(5.)})
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [26]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
       [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

Solution:

In [28]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1
4,4,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2,3


In [32]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'], columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [35]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


####Here we use a similar index and non overlapping columns

In [36]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


##Concatenating along axis

In [38]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [39]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [45]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [47]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default concat works along axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):

In [48]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [50]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

###Challenge!
What would be the difference between: 

pd.concat([s1, s4], axis=1)
and

pd.concat([s1, s4], axis=1, join='inner')

In [52]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

Notice the keys used for each series

In [53]:
result.unstack()

,a,b,f,g
one,0,1,NaN,NaN
two,0,1,NaN,NaN
three,NaN,NaN,5,6


###Challenge!

Try concatenating two data frame objects setting a key for each data frame


If you need to ignore the row names

In [56]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,0.406209,0.134499,-0.736462,1.048186
1,-1.057252,0.250065,-0.238468,0.091275
2,-0.317977,0.986328,-0.392398,0.260411


In [58]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,0.811604,-2.037728,-1.684153
1,-0.544392,-0.561306,-0.452706


In [59]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.406209,0.134499,-0.736462,1.048186
1,-1.057252,0.250065,-0.238468,0.091275
2,-0.317977,0.986328,-0.392398,0.260411
3,-1.684153,0.811604,NaN,-2.037728
4,-0.452706,-0.544392,NaN,-0.561306


## Reshaping

- stack: this “rotates” or pivots from the columns in the data to the rows 
- unstack: this pivots from the rows into the columns

In [62]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [64]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [65]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


## Pivoting - Long to wide 

In [85]:
long_gap = pd.read_csv('long_gap.csv')

In [86]:
long_gap[:10]

,country,Description,country_val
0,Afghanistan,life,37.478833
1,Albania,life,68.432917
2,Algeria,life,59.030167
3,Angola,life,37.883500
4,Argentina,life,69.060417
5,Australia,life,74.662917
6,Austria,life,73.103250
7,Bahrain,life,65.605667
8,Bangladesh,life,49.834083
9,Belgium,life,73.641750


In [87]:
pivoted = long_gap.pivot('country','Description', 'country_val')

In [88]:
pivoted.head()

Description,gdp,life,pop
country,,,
Afghanistan,802.674598,37.478833,15823715.416667
Albania,3255.366633,68.432917,2580249.166667
Algeria,4426.025973,59.030167,19875406.166667
Angola,3607.100529,37.883500,7309390.083333
Argentina,8955.553783,69.060417,28602239.916667


### Challenge!

Can you pivot using unstack and set_index?

In [89]:
unstacked = long_gap.set_index(['country', 'Description']).unstack('Description')

In [90]:
unstacked.head()

country_val                            
Description          gdp       life              pop
country                                             
Afghanistan   802.674598  37.478833  15823715.416667
Albania      3255.366633  68.432917   2580249.166667
Algeria      4426.025973  59.030167  19875406.166667
Angola       3607.100529  37.883500   7309390.083333
Argentina    8955.553783  69.060417  28602239.916667